In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random

In [2]:
find_search_prob = 0.5 
find_wait_prob =  0.2
penality = -3
canReward = 1
rechargeReward = 0
alpha = 0.4
beta = 0.2

In [ ]:
class recyclingRobot:
    def __init__(self):
        self.state = True

    def chooseAction(self):
        if self.state:
            actions = ["serch","wait"]
        else:
            actions = ["search","wait","recharge"]